# Amazon SQS 기초

## SQS의 주요 구성 요소
- 메세지 : SQS에서 전달하는 데이터의 단위
- Producer : 메세지를 생성하고 SQS에 전달하는 주체
- Queue : 메세지를 저장하고 메세지를 컨슈머에게 전달하는 다양한 기능을 담당
- 컨슈머 : 메세지를 받아 처리하고 소비(삭제) 하는 주체
- Dead Letter Queue : 처리에 실패한 메세지를 모아둔 2차 큐
- 액세스 정책 : SQS에 접근할 수 있는 주체에 대한 권한 설정 정책

## SQS의 타입
- 스탠다드
    - 스탠다드 SQS는 순서를 보장하지 않음
    - Standard SQS는 메세지를 여러번 전달할 가능성 존재 : At Least Once Delivery
- FIFO
    - 순서 보장 및 메세지를 단 한번만 전달하도록 보장하나 스탠다드에 비해 떨어지는 성능
    - Exactly Once Delivery
- 즉 Standard 모드를 사용할 때는 멱등성(Indempotency) 확보 필요
    - 멱등성(Indempotency) : 로직이 여러번 수행되더라도 동일한 결과를 보장할 수 있는 성질
    - 예 : 한자리에 대한 기차표가 한 대상에게 여러번 구매되더라도 요금이 여러번 결제되지 않는것

## SQS 메세지
- SQS에서 전달하는 데이터의 단위
- 최대 사이즈 : 256kb, 최대 14일까지 저장 가능
- 메세지는 총 세 가지 상태
    - Stored : Producer가 메세지를 큐에 전달을 완료하여 대기중인 상태
        - 최대 개수 제한 없음
    - In Flight : Consumer가 메세지를 가져와서 처리중인 상태
        - Standard : 약 12만개
        - FIFO : 약 2만개
    - Deleted : Consumer가 메세지 내용을 처리한 후 삭제한 상태
- 메세지의 구성
    - Message Body : 실제 메세지의 내용(String)
        - 최대 256kb(Message Attribute 포함)
        - 크기가 큰 컨텐츠의 경우 S3에 저장 후 버킷/키 정보만을 저장하는 형식으로 전달 가능
    - Message Attribute : Key-Value 형식의 메타데이터로, Body에 포함되지 않는 추가적인 데이터
        - 주로 분류, 필터링, Body를 처리하기 위한 Context등에 활용
            - 예 : Attribute에 영상 처리 알고리즘 명시, 분류를 위한 Tag 정의 등
        - 최대 10개
    - Message System Attributes : AWS 서비스에서 활용하기 위한 메타데이터
        - 현재는 AWS X-ray(모니터링 서비스)를 위한 AWSTraceHeader만 지원
    - ReceiptHandle : 메세지를 삭제하기 위한 키 (아이디로 삭제하는 것이 아님)
    - 기타 정보
        - 리전, Event Source, MD5, Timestamp, 해시 등
    - FIFO의 경우 Message Group ID / Debuplication ID

## Producer
- 메세지는 API를 사용해서 푸쉬 가능
    - 이 때 Queue URL을 알고 있어야 정확한 SQS Queue를 판별하여 메세지 전송 가능
- Queue URL 형식
    - https://sqs.{region}.amazonaws.com/{account-id}/{queue-name}
    - https://sqs.up-east-1.amazonaws.com/1234567/MyQueue
- IAM 엔티티로서 권한이 있거나 리소스 기반 정책으로 권한 확보 필요(sqs:SendMessage)
- 다양한 AWS의 서비스에 이벤트로 연동
    - 기본적으로 연동 가능하거나 이벤트브릿지로 연동

## Consumer
- 권한이 있는 상태에서 주기적으로 API로 Queue에 메세지를 요청(Poll)해서 처리
    - 역시 메세지를 받아오고 삭제하는 권한 필요 (sqs:ReceiveMessage/sqs:DeleteMessage)
- 기본적인 워크플로우
    - Poll, 처리, 삭제
- Visibility Timeout 안에 메세지를 받아 처리하고 완료 시 삭제
    - 삭제하지 않으면 다시 Queue Visibility Timeout 이후 Queue로 반환
    - 혹은 Visibility Timeout을 늘려서 추가 시간을 확보 후 처리
- 두 가지 Polling 방법
    - Short Polling(기본) : 메세지 요청 시 큐의 일부를 검색해서 가능한 메세지를 빠르게 찾아서 전달
        - 메세지를 못 찾아도 응답을 바로 전달
        - False Empty Response 발생 가능
    - Long Polling : 메세지 요청 시 큐 전체를 검색하여 적어도 하나의 메세지를 찾아서 전달
        - 메세지가 없다면 찾을 때까지 기다리거나(생성될 때까지 기다리거나 untill Timeout), 혹은 Timeout을 넘는다면 그때 응답 전달
        - 1~20초 까지 대기시간 설정 가능
        - 장점
            - False Empty Response 방지
            - 요청 횟수 줄이기
    - Batch 처리 가능 : 한번에 최대 10개의 메세지 요청 가능

### Visibility Timeout
- 메세지의 요청 이후 다른 주체가 메세지를 요청할 수 없는 기간
    - 일종의 락
    - 해당 기간동안 다른 주체가 메세지 요청 불가능
- 기본 30초, 최소 0초, 최대 12시간
- Visibility Timeout 기간을 넘어서 처리(삭제)되지 않은 메세지는 자동으로 다른 주체에게 열림(요청 가능)
    - 메세지를 처리하기 충분한 시간이지만 오류/에러 상황에 적절하게 대응할 수 있는 시간 필요
- 기본적으로 큐 단위이나, 메세지 단위로 설정 가능
    - 즉 필요하다면 Visibility Timeout 수정 가능
    - 주의 : Visibility Timeout이 만료되면 큐 맨 뒤로 돌아감

## SQS 보안
- Access Policy : 어떤 주체가 SQS Queue에 접근하여 메세지를 보내거나 가져올 수 있는 리소스 기반 정책
    - 주체에게 권한이 없더라도 권한 부여 가능
    - 예 : IAM 사용자에 아무 권한이 없더라도 Access 정책에 권한이 명시되어 있으면 큐 사용 가능
- 암호화
    - SQS-SSE(Encryption at rest), SQS-KMS 지원 (S3와 거의 동일)
    - 바디만 암호화(메타데이터, 타임스태프 등은 암호화 하지 않음)


### IAM 정책의 종류
- Identity-based policies(자격 증명 기반 정책)
    - 자격증명(IAM유저, 그룹, 역할)에 부여하는 정책
    - 해당 자격증명이 무엇을 할 수 있는지 허용
    - 열쇠 같은 역할
- Resource-based policies(리소스 기반 정책)
    - 리소스(예: S3, SQS, VPC Endpoint)에 부여하는 정책
    - 해당 리소스에 누가 무엇을 할 수 있는지 허용 가능
        - 예 : SQS 대기열에 람다 서비스가 접근 가능
    - 지문인식장치 같은 역할

## Dead Letter Queue
- 시스템 오류로 처리할 수 없는 메세지를 임시로 저장하는 특수한 유형의 메세지 대기열
    - SQS에서 처리하지 못하는 메세지를 담아두어 처리하는 다른 큐
    - 추후 재 처리 가능(Redrice)
- 설정한 재시도 횟수보다 많이 실패했을 경우 DLQ로 전달하도록 설정 가능

## 모니터링
- 기본적으로 클라우드트레일로 API 로깅 가능
- 클라우드워치 기본지표로 다양한 지표 제공
- 주요 메트릭
    - ApproximateAgeOfOldestMessage : (근사값) 가장 오래된 메세지의 나이
    - ApproximateNumberOfMessageNotVisible : (근사값) 현재 In-Flight 중인 메세지
    - ApproximateNumberOfMessageVisible : (근사값) 현재 Stored, 대기중인 메세지
    - NumberOfEmptyReceives : 메세지 요청 시 빈 응답이 전달 된 횟수
    - NumverOfMessagesReceived : 큐에 도착한 메세지 개수
    - SentMessageSize : 큐에 도착한 메세지의 크기